Let's check are we in collab or not!

In [1]:
import sys
IN_COLAB = 'google.colab' in sys.modules


First of all let's connect our drive to save logs properly. It differs between collab environment and non-colab

In [4]:
import pathlib

if IN_COLAB:
    from google.colab import drive
    BASE_PATH = pathlib.Path("/content/drive")
    DRIVE_PATH = BASE_PATH / "MyDrive"
    drive.mount(str(BASE_PATH), force_remount=True)
    
    SAVE_PATH = DRIVE_PATH / "gdl-usi-2024/"
    SAVE_PATH.mkdir(exist_ok=True)
else:
    BASE_PATH = pathlib.Path("experiment_data/")
    SAVE_PATH = BASE_PATH
    

Let's download our toolset for the preparation of our data. Also we need to install some special tooling for our arrangements.

In [5]:
if IN_COLAB:
    !wget https://raw.githubusercontent.com/strategy155/gdl-usi-2024-spring/dev/GDL_data_prep.ipynb --output-document GDL_data_prep.ipynb
    !pip install ipynb
else: 
    pass

Then let's load our data preparation module, and the pandas also. Then we import pandas, download the data, and read it in a usual way.

In [ ]:
from ipynb.fs.defs.GDL_data_prep import get_data, compute_diff_ts, minmax_bound_scaler, check_nans
import pandas as pd

# get_data() # downloading the data to the environment
stocknet_df = pd.read_parquet("/content/drive/MyDrive/merged_raw_dataset.parquet")
stocknet_df.head()

Then we compute differences of our tsdata, then apply the scaler.

In [ ]:
stocknet_diff, invert_minmax = minmax_bound_scaler(
                    compute_diff_ts(stocknet_df)
                )
stocknet_diff.head()

Finally we check the data for the gaps.

In [ ]:
check_nans(stocknet_diff)

Then we decided to remove the poorly formed columns. It concludes the first step of our data preparation.

In [ ]:
stocknet_diff.dropna(axis=1,inplace=True)
stocknet_diff.head()

Let's create our baseline graph, based on the additional information. Let's load our table!

In [ ]:
!wget https://raw.githubusercontent.com/yumoxu/stocknet-dataset/master/StockTable --output-document "stocktable.tsv"

stock_sectors_df = pd.read_table("stocktable.tsv")
stock_sectors_df.head()

Now let's build an adjacency matrix based on that information (everything inside a sector is interconnected). First let's remove a dollar sign from our symbol names in the table, to use the exact order of symbols we have in our stocknet_diff dataset.

In [ ]:
stock_sectors_df["Symbol"] =stock_sectors_df["Symbol"].str.lstrip("$")
stock_sectors_df["Symbol"].head()

We normalize the order of the columns to be the same, so our adjacency will have the exact order of things.

In [ ]:
stock_sectors_df = stock_sectors_df.set_index("Symbol")
stocknet_diff_column_order = stocknet_diff.columns.get_level_values(0).unique()
stock_sectors_df = stock_sectors_df.reindex(stocknet_diff_column_order)
stock_sectors_df.head()

Let's create row-by-row our adjacency matrix, based on the type of the sector of the company, by iterating throught the symbol names of the stocknet_diff.

In [ ]:
import numpy as np

adjacency_rows = []
for stockname_to_sector in stock_sectors_df.itertuples():
  is_sector_equal_row = stockname_to_sector.Sector == stock_sectors_df["Sector"]
  adjacency_rows.append(is_sector_equal_row.astype(int).values)

adjacency_matrix_by_sector = np.stack(adjacency_rows)
adjacency_matrix_by_sector

Let's install the tsl, and the dependencies

In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!pip install -q git+https://github.com/TorchSpatiotemporal/tsl.git


We also need to convert the adjacency matrix to the edge_index format for PyG compatibility.

In [ ]:
from tsl.ops.connectivity import adj_to_edge_index

edge_index_sector = adj_to_edge_index(adjacency_matrix_by_sector)
edge_index_sector

Let's create a torch-based dataset for further training.

In [ ]:
from tsl.data import SpatioTemporalDataset

tsl_stock_df = SpatioTemporalDataset(target=stocknet_diff,
                                     connectivity=edge_index_sector,
                                      horizon=5,
                                      window=12,
                                      stride=1)
print(tsl_stock_df)

Let's checkup the sample

In [ ]:
sample = tsl_stock_df[0]
print(sample)

Let's see the pattern of our data:

In [ ]:
sample.pattern

Let's split our data sequentially. Also, let's setup it in a good way, and perform all the computations!

In [ ]:
from tsl.data.datamodule import (SpatioTemporalDataModule,
                                 TemporalSplitter)
import numpy as np

BATCH_SIZE = 64
VAL_LEN = 0.1
TEST_LEN = 0.1

stocknet_splitter = TemporalSplitter(val_len=VAL_LEN, test_len=TEST_LEN)
stocknet_datamodule = SpatioTemporalDataModule(
    dataset=tsl_stock_df,
    splitter=stocknet_splitter,
    batch_size=BATCH_SIZE,
)

stocknet_datamodule.setup()
print(stocknet_datamodule)

Let's try a simple TTSM model.

In [ ]:
import torch.nn as nn

from tsl.nn.blocks.encoders import RNN
from tsl.nn.layers import NodeEmbedding, DiffConv
from einops.layers.torch import Rearrange  # reshape data with Einstein notation


class TimeThenSpaceModel(nn.Module):
    def __init__(self, input_size: int, n_nodes: int, horizon: int,
                 hidden_size: int = 32,
                 rnn_layers: int = 1,
                 gnn_kernel: int = 2):
        super(TimeThenSpaceModel, self).__init__()

        self.encoder = nn.Linear(input_size, hidden_size)

        self.node_embeddings = NodeEmbedding(n_nodes, hidden_size)

        self.time_nn = RNN(input_size=hidden_size,
                           hidden_size=hidden_size,
                           n_layers=rnn_layers,
                           cell='gru',
                           return_only_last_state=True)

        self.space_nn = DiffConv(in_channels=hidden_size,
                                 out_channels=hidden_size,
                                 k=gnn_kernel)

        self.decoder = nn.Linear(hidden_size, input_size * horizon)
        self.rearrange = Rearrange('b n (t f) -> b t n f', t=horizon)

    def forward(self, x, edge_index, edge_weight):
        # x: [batch time nodes features]
        x_enc = self.encoder(x)  # linear encoder: x_enc = xΘ + b
        x_emb = x_enc + self.node_embeddings()  # add node-identifier embeddings
        h = self.time_nn(x_emb)  # temporal processing: x=[b t n h] -> h=[b n h]
        z = self.space_nn(h, edge_index, edge_weight)  # spatial processing
        x_out = self.decoder(z)  # linear decoder: z=[b n h] -> x_out=[b n t⋅f]
        x_horizon = self.rearrange(x_out) # x_out=[b n t⋅f] -> x_out=[b t n f]
        return x_horizon

Let's actually train our model now!

In [ ]:
hidden_size = 16   #@param
rnn_layers = 1     #@param
gnn_kernel = 2     #@param

input_size = tsl_stock_df.n_channels   # 6 channel
n_nodes = tsl_stock_df.n_nodes         # 81 nodes
horizon = tsl_stock_df.horizon         # 5 time steps

stocknet_ttsm = TimeThenSpaceModel(input_size=input_size,
                           n_nodes=n_nodes,
                           horizon=horizon,
                           hidden_size=hidden_size,
                           rnn_layers=rnn_layers,
                           gnn_kernel=gnn_kernel)
print(stocknet_ttsm)

Let's setup the predictor then.

In [ ]:
from tsl.metrics.torch import MaskedMAE, MaskedMAPE
from tsl.engines import Predictor

stocknet_loss_fn = MaskedMAE()

stocknet_metrics = {'mae': MaskedMAE(),
           'mape': MaskedMAPE(),
           }

# setup predictor
stocknet_ttsm_predictor = Predictor(
    model=stocknet_ttsm,                   # our initialized model
    optim_class=torch.optim.Adam,  # specify optimizer to be used...
    optim_kwargs={'lr': 0.001},    # ...and parameters for its initialization
    loss_fn=stocknet_loss_fn,               # which loss function to be used
    metrics=stocknet_metrics                # metrics to be logged during train/val/test
)

Let's setup our logging facility!

In [ ]:
from pytorch_lightning.loggers import TensorBoardLogger

MODEL_NAME = "stocknet-sector-window-12-forecast-5"
VERSION = 0
MODEL_PATH = SAVE_PATH / MODEL_NAME
stocknet_logger = TensorBoardLogger(save_dir=SAVE_PATH, name=MODEL_NAME, version=VERSION)

Launching tensorboard then!

In [ ]:
%load_ext tensorboard
%tensorboard --logdir $SAVE_PATH


Let's use a lightning trainer for our dirty purposes.

In [ ]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

stocknet_checkpoint_callback = ModelCheckpoint(
    dirpath=MODEL_PATH,
    save_top_k=1,
    monitor='val_mae',
    mode='min',
)

stocknet_ttsm_trainer = pl.Trainer(max_epochs=240,
                     logger=stocknet_logger,
                     limit_train_batches=2000,  # end an epoch after 100 updates
                     callbacks=[stocknet_checkpoint_callback])
checkpoint_paths = [checkpoint for checkpoint in MODEL_PATH.iterdir() if checkpoint.suffix == ".ckpt"]

stocknet_ttsm_trainer.fit(stocknet_ttsm_predictor, datamodule=stocknet_datamodule, ckpt_path=checkpoint_paths[0])

That's nearly perfect. Let's see the test accuracy here.

In [ ]:
stocknet_ttsm_predictor.load_model(stocknet_checkpoint_callback.best_model_path)
stocknet_ttsm_predictor.freeze()

stocknet_ttsm_trainer.test(stocknet_ttsm_predictor, datamodule=stocknet_datamodule);

I've been checking our data, to see if it is finely looking. It looks a bit strangely, so we'll make different scaling strategy later.

In [ ]:
sample = tsl_stock_df[0]
print(sample.y[:,0,:])
stocknet_diff.head(17)

Let's convert our dataset to a proper TSL-based dataset

In [ ]:
from tsl.datasets.prototypes import DatetimeDataset


class StockTSLDataset(DatetimeDataset):
  """
  This is a specially created type of dataset for the Stocknet purposes.
  Probably, we will make it better, later. You load here the stocknet data. More
  details in the original documentation of TSL Datetime dataset, and other ones.
  """

  def __init__(self, target,*args, **kwargs):
    """
    Here the target is a dataframe containing a rightly formatted stocknet dataset
    """
    super(DatetimeDataset, self).__init__(target, sector = None, *args, **kwargs)

  def get_sector_similarity(self, sector_df):
    """
    This function calculates the adjacency matrix by employing the sector data
    and creating non-connected clusters of the stock-nodes
    """
    pass

  def compute_similarity(self, method, **kwargs):
    """
    This function computes similarity between our nodes, if ANY. I'll send you to
    the original TSL doc.
    """
    if method == "sector":
      pass

Let's change the scaler of our data, due to beforementioned facts.


In [ ]:
from ipynb.fs.defs.GDL_data_prep import unit_centered_scaler

unit_stocknet_diff, inverse_unit_centered, invert_values = unit_centered_scaler(
                    compute_diff_ts(stocknet_df)
                )
unit_stocknet_diff.head()

Let's train the normalized data then. We will use the same model, just the data will be different.

In [ ]:
tsl_unit_stock_df = SpatioTemporalDataset(target=unit_stocknet_diff,
                                     connectivity=edge_index_sector,
                                      horizon=5,
                                      window=12,
                                      stride=1)

stocknet_unit_datamodule = SpatioTemporalDataModule(
    dataset=tsl_unit_stock_df,
    splitter=stocknet_splitter,
    batch_size=BATCH_SIZE,
)

stocknet_unit_datamodule.setup()
print(stocknet_unit_datamodule)

In [ ]:
hidden_size = 16   #@param
rnn_layers = 1     #@param
gnn_kernel = 2     #@param

input_size = tsl_unit_stock_df.n_channels   # 6 channel
n_nodes = tsl_unit_stock_df.n_nodes         # 81 nodes
horizon = tsl_unit_stock_df.horizon         # 5 time steps

stocknet_unit_ttsm = TimeThenSpaceModel(input_size=input_size,
                           n_nodes=n_nodes,
                           horizon=horizon,
                           hidden_size=hidden_size,
                           rnn_layers=rnn_layers,
                           gnn_kernel=gnn_kernel)
print(stocknet_unit_ttsm)

In [ ]:

# setup predictor
stocknet_unit_ttsm_predictor = Predictor(
    model=stocknet_unit_ttsm,                   # our initialized model
    optim_class=torch.optim.Adam,  # specify optimizer to be used...
    optim_kwargs={'lr': 0.001},    # ...and parameters for its initialization
    loss_fn=stocknet_loss_fn,               # which loss function to be used
    metrics=stocknet_metrics                # metrics to be logged during train/val/test
)

MODEL_NAME = "stocknet-sector-unit-norm-window-12-forecast-5"
VERSION = 0
MODEL_PATH = SAVE_PATH / MODEL_NAME
stocknet_unit_logger = TensorBoardLogger(save_dir=SAVE_PATH, name=MODEL_NAME, version=VERSION)

Getting our logger ready

In [ ]:
%load_ext tensorboard
%tensorboard --logdir $SAVE_PATH


In [ ]:
stocknet_unit_checkpoint_callback = ModelCheckpoint(
    dirpath=MODEL_PATH,
    save_top_k=1,
    monitor='val_mae',
    mode='min',
)

MODEL_PATH.mkdir(exist_ok=True)

stocknet_unit_ttsm_trainer = pl.Trainer(max_epochs=230,
                     logger=stocknet_unit_logger,
                     limit_train_batches=2000,  # end an epoch after 100 updates
                     callbacks=[stocknet_unit_checkpoint_callback])
checkpoint_paths = [checkpoint for checkpoint in MODEL_PATH.iterdir() if checkpoint.suffix == ".ckpt"]
current_ckpt_path = checkpoint_paths[0] if checkpoint_paths else None

stocknet_unit_ttsm_trainer.fit(stocknet_unit_ttsm_predictor, datamodule=stocknet_unit_datamodule, ckpt_path=None)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:653: Checkpoint directory /content/drive/MyDrive/gdl-usi-2024/stocknet-sector-unit-norm-window-12-forecast-5 exists and is not empty.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name          | Type               | Params
-----------------------------------------------------
0 | loss_fn       | MaskedMAE          | 0     
1 | train_metrics | MetricCollection   | 0     
2 | val_metrics   | MetricCollection   | 0     
3 | test_metrics  | MetricCollection   | 0     
4 | model         | TimeThenS

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (15) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...
